# Functions and imports

In [1]:
%reload_ext autoreload
%autoreload 2

from copy import deepcopy
from datetime import datetime, timedelta
from file_exchange import config, network
from file_exchange.network import Network
from file_exchange.peers_view import PeersView
from file_exchange.user import User
from gevent.event import AsyncResult
from gevent.pool import Group
from gevent.select import select
import gevent
import json
import numpy as np
import os
import pandas as pd
import pickle
import random
import shutil
import sys
import time

def get_users_pair(users):
    u1 = random.choice(users)
    u2 = random.choice(users)
    while u1 == u2:
        u2 = random.choice(users)
    return u1, u2

def save_conf(conf):
    with open(conf['output_dir']+"conf.json", "w") as f:
        conf2 = deepcopy(conf)
        conf2['period'] = str(conf2['period'])
        json.dump(conf2, f)
def load_conf(path):
    f = open(path+'conf.json', 'r')
    conf = json.load(f)
    conf['period'] = datetime.strptime(conf['period'], "%H:%M:%S") - \
                      datetime.strptime("00:00", "%H:%M")
    return conf        

def init_dir(path):
    if path is None:
        return
    if os.path.isdir(path):
        if os.listdir(path):
            ans = input("Remove previous content of "+path+"? (y/N) ")
            if ans == 'Y' or ans == 'y' or ans == 'o' or ans == 'O':
                shutil.rmtree(path)
    if not os.path.isdir(path):
        os.makedirs(path)
        
def run_experiment(conf):
    init_dir(conf['output_dir'])

    # Initializing stuff
    address_book = PeersView(conf)
    net = Network()
    users = [None] * conf['n_users']
    for i in range(conf['n_users']):
        users[i] = User(address_book, net, conf)

    # Picking users for file exchange
    users_pairs = set()
    print("Will do {} file exchanges".format(conf['n_file_exchanges']))
    while len(users_pairs) < conf['n_file_exchanges']:
        u1, u2 = get_users_pair(users)
        users_pairs.add((u1, u2))

    # Start greenlets
    for u in users:
        u.start()
    print("Started all users' threads")
    

    # Wait a bit for everyone to register to address_book
    gevent.sleep(conf['period'].total_seconds())
    # Init file exchanges
    files_info=[]
    for u1, u2 in users_pairs:
        try:
            print("Starting file exchange between {} and {}".format(
                u1.name, u2. name), flush=True)
            file_info = u2.init_file_receive(u1)
            files_info.append(file_info)
        except:
            print("Failed starting file exchange between {} and {}".format(
                u1.name, u2. name), flush=True)
            pass
    n_file_exchanges = len(files_info)
    
    # Wait for all greenlets to terminate
    print("Waiting for experiment termination")
    gevent.joinall(users)
    
    # Save config
    if conf['output_dir'] is not None:
        save_conf(conf)
        with open(conf['output_dir']+"files_info.bin", 'w') as f:
            pickle.dump(files_info, f)

    print("Done")

# File sharing

In [ ]:
conf = config.default
conf['n_users'] = 10
conf['n_rounds'] = 30
conf['period'] = timedelta(seconds=3)
conf['output_dir'] = 'data/file_exchange_random_connected/'
conf['send_strategy'] = 'random_connected'
conf['n_file_exchanges'] = 10
print(conf)

run_experiment(conf)

{'output_dir': 'data/file_exchange_random_connected/', 'n_rounds': 30, 'ack_size': 0, 'do_monitor': True, 'n_file_exchanges': 10, 'n_file_chunks': 10, 'gossip_size': 20, 'file_size': 11, 'layer_threshold': 0.001, 'period': datetime.timedelta(0, 3), 'ping_time': 0.1, 'minimum_node_availability': 0, 'header_size': 0.1, 'chunk_max_size': 2, 'send_strategy': 'random_connected', 'n_users': 10, 'bandwidth': 10, 'n_layers': 3}
Remove previous content of data/file_exchange_random_connected/? (y/N) y
Will do 10 file exchanges
Started all users' threads
Starting file exchange between Moira and Mazie
Device ff6moi645nf1 (owned by Moira) will send file z6pzso
Starting file exchange between Evelyne and Lyndsey
[build route] No more available nodes: early abort.
Failed starting file exchange between Evelyne and Lyndsey
Starting file exchange between Bronwyn and Evelyne
Failed starting file exchange between Bronwyn and Evelyne
Starting file exchange between Shawnda and Cordia
Device af1qah8g1ifi (own

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/gevent/greenlet.py", line 536, in run
    result = self._run(*self.args, **self.kwargs)
  File "/home/adrien/Cloud/These/p2p-private-cloud/code/notebooks/file_exchange/device.py", line 56, in _run
    while not self.online.wait(0.1):
  File "/usr/local/lib/python3.5/dist-packages/gevent/event.py", line 194, in wait
    def wait(self, timeout=None):
KeyboardInterrupt
Thu Apr 26 14:34:30 2018 <Device at 0x7ff4a64a3c28> failed with KeyboardInterrupt



Failed starting file exchange between Bronwyn and Lyndsey
Starting file exchange between Karlene and Barry
Device km6gv32e02xe (owned by Karlene) will send file ftxcgi
Starting file exchange between Cordia and Evelyne


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/gevent/greenlet.py", line 536, in run
    result = self._run(*self.args, **self.kwargs)
  File "/home/adrien/Cloud/These/p2p-private-cloud/code/notebooks/file_exchange/device.py", line 163, in _run
    gevent.sleep(0)
  File "/usr/local/lib/python3.5/dist-packages/gevent/hub.py", line 166, in sleep
    loop.run_callback(waiter.switch)
KeyboardInterrupt
Thu Apr 26 14:34:32 2018 <Device at 0x7ff4a6488470> failed with KeyboardInterrupt



In [ ]:
# Interpretation

In [2]:
def fetch_devices_data(conf):
    df = pd.DataFrame()
    for fn in os.listdir(conf['output_dir']):
        if not fn.endswith(".csv"):
            continue
            
        filepath = conf['output_dir']+fn
        try:
            # df=pd.read_csv(filepath, parse_dates=['Timestamp'],
            # index_col=['Timestamp'])
            df2 = pd.read_csv(filepath)
        except:
            print("Failed loading {}: {}".format(fn, sys.exc_info()))
            pass
        else:
            if(df.empty):
                df = df2
            else:
                df = df.append(df2,)
            #print("Loaded dataframe {}.".format(fn))

    df['t'] -= min(df['t'])
    df = df.sort_values('t')
    return df

def fetch_experiment_data(path):
    conf = load_conf(path)
    with open(conf['output_dir']+"files_info.bin", 'w') as f:
        files_info = pickle.load(f)
    df = fetch_devices_data(conf)  
    return conf, files_info, df
conf, files_info, df = fetch_experiment_data("data/file_exchange_random_connected/")

UnsupportedOperation: not readable

In [ ]:
np.linspace()